In [11]:
import plotly.graph_objects as go
import numpy as np
import random

In [12]:
all_obstacles = []
for i in range(30):
    x = random.randint(30, 6000)
    y = random.randint(30, 6000)
    z = 300
    r = random.randint(5, 151)
    all_obstacles.append([x, y, z, r])

In [13]:
all_obstacles

[[3424, 1260, 300, 120],
 [3463, 791, 300, 122],
 [4754, 2948, 300, 15],
 [1610, 3517, 300, 42],
 [1351, 3181, 300, 24],
 [5776, 2478, 300, 113],
 [5035, 4154, 300, 111],
 [3154, 2770, 300, 125],
 [1242, 2525, 300, 122],
 [3459, 4133, 300, 148],
 [5515, 2503, 300, 91],
 [3477, 37, 300, 74],
 [194, 3486, 300, 8],
 [5769, 118, 300, 43],
 [65, 5882, 300, 25],
 [1183, 1971, 300, 8],
 [1135, 2918, 300, 70],
 [1520, 5829, 300, 68],
 [2638, 2510, 300, 120],
 [3066, 4251, 300, 130],
 [4798, 3691, 300, 99],
 [1374, 5071, 300, 141],
 [4220, 5858, 300, 27],
 [141, 5468, 300, 38],
 [5972, 2268, 300, 25],
 [3177, 3815, 300, 73],
 [1084, 3819, 300, 22],
 [2190, 1834, 300, 36],
 [371, 3617, 300, 68],
 [2458, 2983, 300, 139]]

In [14]:
x = [i[0] for i in all_obstacles]
y = [i[1] for i in all_obstacles]
z = [i[2] for i in all_obstacles]

In [15]:
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers', marker=dict(
        size=7,
        color=z,                  
        opacity=0.8
    ))])
fig.show()
